In [1]:
import pandas as pd
import numpy as np
import os 
import pickle
from collections import Counter
from tqdm import tqdm 
pd.options.display.max_columns = None

# Region selection
Checking the data from the [given](https://www.kaggle.com/datasets/andrewmvd/spotify-playlists?select=spotify_dataset.csv) database.

In [2]:
data = pd.read_csv('../data/charts.csv', sep = ',', on_bad_lines='skip')
print(data.shape)
data.head()

(26173514, 9)


,title,rank,date,artist,url,region,chart,trend,streams
0,Chantaje (feat. Maluma),1,2017-01-01,Shakira,https://open.spotify.com/track/6mICuAdrwEjh6Y6...,Argentina,top200,SAME_POSITION,253019.0
1,Vente Pa' Ca (feat. Maluma),2,2017-01-01,Ricky Martin,https://open.spotify.com/track/7DM4BPaS7uofFul...,Argentina,top200,MOVE_UP,223988.0
2,Reggaetón Lento (Bailemos),3,2017-01-01,CNCO,https://open.spotify.com/track/3AEZUABDXNtecAO...,Argentina,top200,MOVE_DOWN,210943.0
3,Safari,4,2017-01-01,"J Balvin, Pharrell Williams, BIA, Sky",https://open.spotify.com/track/6rQSrBHf7HlZjtc...,Argentina,top200,SAME_POSITION,173865.0
4,Shaky Shaky,5,2017-01-01,Daddy Yankee,https://open.spotify.com/track/58IL315gMSTD37D...,Argentina,top200,MOVE_UP,153956.0


In [3]:
data[['region', 'chart']].value_counts()

region         chart  
Brazil         top200     364516
Argentina      top200     364385
United States  top200     364184
Austria        top200     363925
Australia      top200     362189
                           ...  
India          viral50     51688
Ukraine        viral50     26778
Russia         viral50     26069
South Korea    viral50     16700
Luxembourg     top200       8962
Length: 139, dtype: int64

For my analysis, I will consider one of the regions given by the dataset. In this case, USA.

In [4]:
data[data['region'] == 'United States'][['region', 'chart']].value_counts()

region         chart  
United States  top200     364184
               viral50     90873
dtype: int64

In [5]:
data_usa = data[(data['region'] == 'United States')]

# EDA

Total number of rows and columns of the dataframe

In [6]:
data_usa.shape

(455057, 9)

Counting duplicated rows

In [7]:
data_usa.duplicated().sum()

0

Counting null values

In [8]:
data_usa.isnull().sum() / data_usa.shape[0]

title      0.000000
rank       0.000000
date       0.000000
artist     0.000000
url        0.000000
region     0.000000
chart      0.000000
trend      0.000000
streams    0.199696
dtype: float64

In [9]:
data_usa.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 455057 entries, 8246 to 26173414
Data columns (total 9 columns):
 #   Column   Non-Null Count   Dtype  
---  ------   --------------   -----  
 0   title    455057 non-null  object 
 1   rank     455057 non-null  int64  
 2   date     455057 non-null  object 
 3   artist   455057 non-null  object 
 4   url      455057 non-null  object 
 5   region   455057 non-null  object 
 6   chart    455057 non-null  object 
 7   trend    455057 non-null  object 
 8   streams  364184 non-null  float64
dtypes: float64(1), int64(1), object(7)
memory usage: 34.7+ MB


The only null columns available on the dataset are on the `streams` column. This one I will enrich it with api info.

Main statistic values for numeric columns

In [10]:
data_usa.describe().T

,count,mean,std,min,25%,50%,75%,max
rank,455057.0,85.525380,60.065784,1.0,32.0,76.0,138.00,200.0
streams,364184.0,390275.482734,278800.876862,122488.0,236157.0,294791.5,434688.25,6146233.0


Main statistic values for categorical columns

In [11]:
data_usa.describe(include='object').T

,count,unique,top,freq
title,455057,11842,goosebumps,1778
date,455057,1826,2017-01-01,250
artist,455057,5213,Juice WRLD,10839
url,455057,14650,https://open.spotify.com/track/6gBFPUFcJLzWGx4...,1778
region,455057,1,United States,455057
chart,455057,2,top200,364184
trend,455057,4,MOVE_DOWN,195116


# Adding new columns

Adding columns for top ranking and weeks in charts

In [12]:
data_usa = data_usa.merge(data_usa[['url', 'chart']].value_counts().reset_index().rename({0:'week_in_charts'}, axis = 1), on=['url', 'chart'], indicator=True, how = 'left')
data_usa['_merge'].value_counts()

both          455057
left_only          0
right_only         0
Name: _merge, dtype: int64

In [13]:
data_usa.drop(['_merge'], axis = 1, inplace=True)
data_usa = data_usa.merge(data_usa.sort_values('rank').drop_duplicates(['url','chart'])[['url', 'rank', 'chart']].rename({'rank':'times_in_charts'}, axis = 1), on=['url', 'chart'], 
    indicator=True, how = 'left')
data_usa['_merge'].value_counts()

both          455057
left_only          0
right_only         0
Name: _merge, dtype: int64

In [14]:
data_usa.drop(['_merge'], axis = 1, inplace=True)

# Export
I will export the whole dataset and generate a dataset with the tracks and artist without duplicated rows.

In [15]:
with open('../data/pickle/playlist.pickle', 'wb') as data:
    pickle.dump(data_usa, data)

# Generate clean artist - track data
I will now get a unique list from the track - artist relation, in order to make api calls.

In [16]:
data_usa.head()

,title,rank,date,artist,url,region,chart,trend,streams,week_in_charts,times_in_charts
0,Bad and Boujee (feat. Lil Uzi Vert),1,2017-01-01,Migos,https://open.spotify.com/track/4Km5HrUvYTaSUfi...,United States,top200,SAME_POSITION,1371493.0,429,1
1,Fake Love,2,2017-01-01,Drake,https://open.spotify.com/track/343YBumqHu19cGo...,United States,top200,SAME_POSITION,1180074.0,455,2
2,Starboy,3,2017-01-01,"The Weeknd, Daft Punk",https://open.spotify.com/track/5aAx2yezTd8zXrk...,United States,top200,SAME_POSITION,1064351.0,451,3
3,Closer,4,2017-01-01,"The Chainsmokers, Halsey",https://open.spotify.com/track/7BKLCZ1jbUBVqRi...,United States,top200,SAME_POSITION,1010492.0,538,4
4,Black Beatles,5,2017-01-01,"Rae Sremmurd, Gucci Mane",https://open.spotify.com/track/6fujklziTHa8uoM...,United States,top200,SAME_POSITION,874289.0,240,5


In [17]:
data_usa = data_usa[['title', 'artist', 'url']]
print(data_usa.shape)
data_usa.head()

(455057, 3)


,title,artist,url
0,Bad and Boujee (feat. Lil Uzi Vert),Migos,https://open.spotify.com/track/4Km5HrUvYTaSUfi...
1,Fake Love,Drake,https://open.spotify.com/track/343YBumqHu19cGo...
2,Starboy,"The Weeknd, Daft Punk",https://open.spotify.com/track/5aAx2yezTd8zXrk...
3,Closer,"The Chainsmokers, Halsey",https://open.spotify.com/track/7BKLCZ1jbUBVqRi...
4,Black Beatles,"Rae Sremmurd, Gucci Mane",https://open.spotify.com/track/6fujklziTHa8uoM...


In [18]:
print('total artists:',len(data_usa['artist'].tolist()),'; total songs (with unique names):',len(data_usa['title'].tolist()),
    '; unique artists:',len(data_usa['artist'].unique().tolist()),'; unique songs (with unique names):',len(data_usa['title'].unique().tolist())
)

total artists: 455057 ; total songs (with unique names): 455057 ; unique artists: 5213 ; unique songs (with unique names): 11842


In [19]:
data_usa.drop_duplicates(inplace=True)
print(data_usa.shape)
data_usa.head()

(14663, 3)


,title,artist,url
0,Bad and Boujee (feat. Lil Uzi Vert),Migos,https://open.spotify.com/track/4Km5HrUvYTaSUfi...
1,Fake Love,Drake,https://open.spotify.com/track/343YBumqHu19cGo...
2,Starboy,"The Weeknd, Daft Punk",https://open.spotify.com/track/5aAx2yezTd8zXrk...
3,Closer,"The Chainsmokers, Halsey",https://open.spotify.com/track/7BKLCZ1jbUBVqRi...
4,Black Beatles,"Rae Sremmurd, Gucci Mane",https://open.spotify.com/track/6fujklziTHa8uoM...


In [20]:
print('unique artists:',len(data_usa['artist'].unique().tolist()),'; unique songs (with unique names):',len(data_usa['title'].unique().tolist()))

unique artists: 5213 ; unique songs (with unique names): 11842


making all `artist` lowercase, removing all spaces and weird characters

In [21]:
data_usa['artist_clean'] = data_usa['artist'].str.strip().str.lower()
print(len(data_usa['artist_clean'].tolist()), len(data_usa['artist'].unique().tolist()))

14663 5213


doing the same with the `tracks`

In [22]:
data_usa['track_clean'] = data_usa['title'].str.strip().str.lower()
print(len(data_usa['track_clean'].unique().tolist()), len(data_usa['title'].unique().tolist()))

11635 11842


# Conversion
I will save the input file as .pkl as the original one is too big for github.

In [23]:
data_final = data_usa.drop_duplicates(subset=['artist_clean', 'track_clean', 'url'])
print(data_final.shape)
data_final.head()

(14659, 5)


,title,artist,url,artist_clean,track_clean
0,Bad and Boujee (feat. Lil Uzi Vert),Migos,https://open.spotify.com/track/4Km5HrUvYTaSUfi...,migos,bad and boujee (feat. lil uzi vert)
1,Fake Love,Drake,https://open.spotify.com/track/343YBumqHu19cGo...,drake,fake love
2,Starboy,"The Weeknd, Daft Punk",https://open.spotify.com/track/5aAx2yezTd8zXrk...,"the weeknd, daft punk",starboy
3,Closer,"The Chainsmokers, Halsey",https://open.spotify.com/track/7BKLCZ1jbUBVqRi...,"the chainsmokers, halsey",closer
4,Black Beatles,"Rae Sremmurd, Gucci Mane",https://open.spotify.com/track/6fujklziTHa8uoM...,"rae sremmurd, gucci mane",black beatles


In [24]:
with open('../data/pickle/artist.pickle', 'wb') as data_nowplaying:
    pickle.dump(data_final[['url', 'artist_clean', 'track_clean']], data_nowplaying)